## Step A

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import collections

In [ ]:
query_imgs = ['0','1','11','19','24','25','26']
train_imgs = ['scenes/e1.png','scenes/e2.png','scenes/e3.png','scenes/e4.png','scenes/e5.png']
true_imgs = {
    'scenes/e1.png':{'0':1,'11':1},
    'scenes/e2.png':{'24':1,'25':1,'26':1},
    'scenes/e3.png':{'0':1,'1':1,'11':1},
    'scenes/e4.png':{'0':1,'11':1,'25':1,'26':1},
    'scenes/e5.png':{'19':1,'25':1},
}

In [ ]:
MIN_MATCH_COUNT = 30
LOWE_COEFF = 0.5
COLOR_T = 50
COLORS = False

In [ ]:
sift = cv2.xfeatures2d.SIFT_create()

for train_img in train_imgs:

    train = cv2.imread(train_img,0)
    train_bgr = cv2.imread(train_img)
    if COLORS == True:
        train3 = cv2.cvtColor(train_bgr,cv2.COLOR_BGR2RGB)
        train_rgb = np.zeros(train_bgr.shape, train_bgr.dtype)
        for y in range(train3.shape[0]):
            for x in range(train3.shape[1]):
                for c in range(train3.shape[2]):
                    train_rgb[y,x,c] = np.clip(0.5*train3[y,x,c],0, 255)
    else:
        train_rgb = cv2.cvtColor(train,cv2.COLOR_GRAY2RGB)
    
    kp_train, des_train = sift.detectAndCompute(train,None)
    
    print("Scene: "+train_img+"\n")
    
    global_matches = {}
    recognised = {}
    
    for query_img in query_imgs:
        
        file = 'models/' + query_img + '.jpg'
        query = cv2.imread(file,0)

        kp_query, des_query = sift.detectAndCompute(query,None)

        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks = 50)

#        flann = cv2.FlannBasedMatcher(index_params, search_params)
        flann = cv2.BFMatcher()

        matches = flann.knnMatch(des_query,des_train,k=2)

        good_matches = []
        for m,n in matches:
            if m.distance < LOWE_COEFF * n.distance:
                good_matches.append(m)
                
        global_matches[query_img] = [len(good_matches),good_matches,kp_query]
    
    sorted_global_matches = collections.OrderedDict(sorted(global_matches.items(), key=lambda item: item[1][0],reverse=True))
    
    for k,v in sorted_global_matches.items():

        if v[0] > MIN_MATCH_COUNT:
            
            file = 'models/' + k + '.jpg'
            query_bgr = cv2.imread(file)
            
            src_pts = np.float32([ v[2][m.queryIdx].pt for m in v[1]]).reshape(-1,1,2)
            dst_pts = np.float32([ kp_train[m.trainIdx].pt for m in v[1]]).reshape(-1,1,2)
            M, _ = cv2.findHomography(src_pts,dst_pts,cv2.RANSAC,5.0)
            h,w,d = query_bgr.shape
            pts = np.float32([[0,0],[0,h-1],[w-1,h-1],[w-1,0]]).reshape(-1,1,2)
            dst = cv2.perspectiveTransform(pts,M)

        
            xCenter, yCenter = ((dst[0][0][i]+ dst[2][0][i])/2 for i in (0,1))
                                  
            x_min = int(max(np.min(dst,axis=0)[0][0],0))
            y_min = int(max(np.min(dst,axis=0)[0][1],0))
            x_max = int(min(np.max(dst,axis=0)[0][0],train.shape[1]))
            y_max = int(min(np.max(dst,axis=0)[0][1],train.shape[0]))

            train_crop = train_bgr[y_min:y_max,x_min:x_max]
            train_color = train_crop.mean(axis=0).mean(axis=0)
            query_color = query_bgr.mean(axis=0).mean(axis=0)
            color_diff = np.sqrt(np.sum([value**2 for value in abs(query_color-train_color)]))
            
            a = True 
            if color_diff<COLOR_T :
                for r,corners in recognised.items():
                    if xCenter>corners[0][0][0] and xCenter<corners[3][0][0] and yCenter>corners[0][0][1] and yCenter<corners[1][0][1]:
                        a = False
                        break
                if a : recognised[k] = dst

                
            
    for k,v in recognised.items():
        train_rgb = cv2.polylines(train_rgb,[np.int32(v)],True,(0,255,0),3, cv2.LINE_AA)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(train_rgb, k,\
                        (int((v[3][0][0]-v[0][0][0])/4+v[0][0][0]),int((v[1][0][1]-v[0][0][1])*0.67+v[0][0][1])),\
                        font, 5, (0,255,0), 10, cv2.LINE_AA)
    plt.imshow(train_rgb),plt.show();
